In [1]:
%load_ext autoreload
%autoreload 2


import torch

import torch.nn as nn
from torch.distributions import Categorical, Poisson, MixtureSameFamily
from matplotlib import pyplot as plt
import pandas as pd

# Cd to code
import os
import sys
os.chdir('/cluster/home/kheuto01/code/prob_diff_topk')
sys.path.append('/cluster/home/kheuto01/code/prob_diff_topk')

from datasets import example_datasets, to_numpy
import os
import torch
import numpy as np
import pandas as pd
import time
from functools import partial
from distributions import QuantizedNormal
from torch_models import  MixtureOfTruncNormModel, torch_bpr_uncurried, deterministic_bpr, SpatialWaves
#from torch_perturb.torch_pert_topk import PerturbedTopK
from metrics import top_k_onehot_indicator
from torch_training import train_epoch_largesynth
from torch_perturb.perturbations import perturbed
from torch_models import NegativeBinomialRegressionModel, MixtureOfTruncNormModel, torch_bpr_uncurried, deterministic_bpr

2024-10-22 16:05:38.855369: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-22 16:05:38.909267: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-22 16:05:38.909312: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-22 16:05:38.910620: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-22 16:05:38.920345: I tensorflow/core/platform/cpu_feature_guar

In [2]:
train_X_df = pd.read_csv('mass_train_x.csv', index_col=[0,1])
train_Y_df = pd.read_csv('mass_train_y.csv', index_col=[0,1])

In [3]:
import pandas as pd
import numpy as np

def convert_df_to_3d_array(df):
    # Ensure the DataFrame has a MultiIndex with 'geoid' and 'timestep'
    if not isinstance(df.index, pd.MultiIndex) or set(df.index.names) != {'geoid', 'timestep'}:
        raise ValueError("DataFrame must have a MultiIndex with levels 'geoid' and 'timestep'")

    # Get unique geoids and timesteps, sorted
    geoids = sorted(df.index.get_level_values('geoid').unique())
    timesteps = sorted(df.index.get_level_values('timestep').unique())

    # Create a mapping of geoids to indices
    geoid_to_idx = {geoid: idx for idx, geoid in enumerate(geoids)}

    # Initialize the 3D array
    num_timesteps = len(timesteps)
    num_locations = len(geoids)
    num_features = len(df.columns)
    X = np.zeros((num_timesteps, num_locations, num_features))

    # Fill the 3D array
    for (geoid, timestep), row in df.iterrows():
        t_idx = timesteps.index(timestep)
        g_idx = geoid_to_idx[geoid]
        X[t_idx, g_idx, :] = row.values

    return X, geoids, timesteps

def convert_y_df_to_2d_array(y_df, geoids, timesteps):
    # Ensure the DataFrame has a MultiIndex with 'geoid' and 'timestep'
    if not isinstance(y_df.index, pd.MultiIndex) or set(y_df.index.names) != {'geoid', 'timestep'}:
        raise ValueError("DataFrame must have a MultiIndex with levels 'geoid' and 'timestep'")

    # Initialize the 2D array
    num_timesteps = len(timesteps)
    num_locations = len(geoids)
    y = np.zeros((num_timesteps, num_locations))

    # Create a mapping of geoids to indices
    geoid_to_idx = {geoid: idx for idx, geoid in enumerate(geoids)}

    # Fill the 2D array
    for (geoid, timestep), value in y_df.iloc[:, 0].items():
        t_idx = timesteps.index(timestep)
        g_idx = geoid_to_idx[geoid]
        y[t_idx, g_idx] = value

    return y


In [4]:
train_X, geoids, timesteps = convert_df_to_3d_array(train_X_df.drop(columns='timestep.1'))
time_arr = np.array([timesteps] * len(geoids)).T
train_y = convert_y_df_to_2d_array(train_Y_df, geoids, timesteps)

In [5]:
X_tensor = torch.tensor(train_X, dtype=torch.float32)
y_tensor = torch.tensor(train_y, dtype=torch.float32)
time_tensor = torch.tensor(time_arr, dtype=torch.float32)

In [28]:
model =NegativeBinomialRegressionModel(num_locations=len(geoids), num_fixed_effects=train_X.shape[2])
for param in model.parameters():
    param.requires_grad_(True)

In [29]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [30]:
M_score_func =  20
M_action = 20
top_k_func = partial(top_k_onehot_indicator, k=100)
perturbed_top_K_func = perturbed(top_k_func, sigma=0.01, num_samples=50)
losses, bprs, nlls, times = [], [], [], []

In [31]:
device = 'cuda'
X_tensor = X_tensor.to(device)
y_tensor = y_tensor.to(device)
time_tensor = time_tensor.to(device)
model.to(device)
model.log_likelihood(y_tensor, X_tensor, time_tensor)

tensor(-3.6982, device='cuda:0', grad_fn=<AddBackward0>)

In [32]:
def train_epoch_neg_binom(model, optimizer, K, threshold,
                           M_score_func, feat_TSF,
                           time_T, train_y_TS,
                           perturbed_top_K_func, bpr_weight, nll_weight, update=True):

    optimizer.zero_grad()
    dist = model(feat_TSF, time_T)
    
    y_sample_TMS = dist.sample((M_score_func,)).permute(1, 0, 2)
    y_sample_action_TMS = y_sample_TMS

    ratio_rating_TMS = y_sample_action_TMS/y_sample_action_TMS.sum(dim=-1, keepdim=True)
    ratio_rating_TS =  ratio_rating_TMS.mean(dim=1)
    ratio_rating_TS.requires_grad_(True)

    def get_log_probs_baked(param):
        distribution = model.build_from_single_tensor(param, feat_TSF, time_T)
        log_probs_TMS = distribution.log_prob(y_sample_TMS.permute(1, 0, 2)).permute(1, 0, 2)

        return log_probs_TMS
    
    jac_TMSP = torch.autograd.functional.jacobian(get_log_probs_baked, (model.params_to_single_tensor()), strategy='forward-mode', vectorize=True)

    score_func_estimator_TMSP = jac_TMSP * ratio_rating_TMS.unsqueeze(-1)
    score_func_estimator_TSP = score_func_estimator_TMSP.mean(dim=1)    

    # get gradient of negative bpr_t  with respect to ratio rating_TS
    print(f'train_y_TS size: {train_y_TS.shape}')
    print(f'ratio_rating_TS size: {ratio_rating_TS.shape}')
    positive_bpr_T = torch_bpr_uncurried(ratio_rating_TS, torch.tensor(train_y_TS), K=K, perturbed_top_K_func=perturbed_top_K_func)
    if nll_weight>0:
        bpr_threshold_diff_T = positive_bpr_T - threshold
        violate_threshold_flag = bpr_threshold_diff_T < 0
        negative_bpr_loss = torch.mean(-bpr_threshold_diff_T*violate_threshold_flag)
    else:
        negative_bpr_loss = torch.mean(-positive_bpr_T)
    
    nll = -model.log_likelihood(train_y_TS, feat_TSF, time_T)

    loss = bpr_weight*negative_bpr_loss + nll_weight*nll
    loss.backward()
    print([param.grad for param in model.parameters()])
    loss_grad_TS = ratio_rating_TS.grad

    gradient_TSP = score_func_estimator_TSP * torch.unsqueeze(loss_grad_TS, -1)
    gradient_P = torch.sum(gradient_TSP, dim=[0,1])

    gradient_tuple = model.single_tensor_to_params(gradient_P)

    for param, gradient in zip(model.parameters(), gradient_tuple):
        if nll_weight>0:
            gradient = gradient + param.grad
        param.grad = gradient
    if update:
        optimizer.step()

    deterministic_bpr_T = deterministic_bpr(ratio_rating_TS, torch.tensor(train_y_TS), K=K)
    det_bpr =torch.mean(deterministic_bpr_T)

    print(f'det bpr: {det_bpr}')
    print(f'Pert bpr: {torch.mean(positive_bpr_T)}')
    print(f'nll: {nll}')
    print(f'Loss: {loss}')
    sys.stdout.flush()

    return loss.detach(), det_bpr.detach(), nll.detach(), model

In [33]:
for epoch in range(200):
    print(f'EPOCH: {epoch}')
    start = time.time()
    loss, bpr, nll, model = train_epoch_neg_binom(model, optimizer, 100, 1,
                                        M_score_func,X_tensor, time_tensor,
                                        y_tensor,
                                        perturbed_top_K_func, 1, 
                                        1)
    end = time.time()
    elapsed = end - start
    losses.append(loss)
    bprs.append(bpr)
    nlls.append(nll)
    times.append(elapsed)

EPOCH: 0
train_y_TS size: torch.Size([14, 1620])
ratio_rating_TS size: torch.Size([14, 1620])
[tensor([-0.0047], device='cuda:0'), tensor([-0.1973,  0.3331, -0.0032, -0.0034, -0.0025, -0.0034, -0.0035, -0.0059,
        -0.0049, -0.0065, -0.0049, -0.0036, -0.0078, -0.0103, -0.0093, -0.0103,
        -0.0120], device='cuda:0'), tensor([-5.1761e-05,  1.3921e-04,  2.7337e-04,  ..., -7.6843e-05,
        -1.3714e-03, -6.8948e-04], device='cuda:0'), tensor([ 0.0006, -0.0005, -0.0010,  ..., -0.0007,  0.0058,  0.0031],
       device='cuda:0'), tensor([-0.5004], device='cuda:0'), tensor([-3.4042], device='cuda:0'), tensor([1.9754], device='cuda:0'), tensor([0.2989], device='cuda:0')]
det bpr: 0.20083190500736237
Pert bpr: 0.1906263530254364
nll: 3.6982064247131348
Loss: 4.507580280303955
EPOCH: 1
train_y_TS size: torch.Size([14, 1620])
ratio_rating_TS size: torch.Size([14, 1620])
[tensor([-0.0045], device='cuda:0'), tensor([-0.1895,  0.3200, -0.0031, -0.0033, -0.0024, -0.0033, -0.0033, -0.0058,
 

/tmp/ipykernel_47034/3412357786.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  positive_bpr_T = torch_bpr_uncurried(ratio_rating_TS, torch.tensor(train_y_TS), K=K, perturbed_top_K_func=perturbed_top_K_func)
/tmp/ipykernel_47034/3412357786.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  deterministic_bpr_T = deterministic_bpr(ratio_rating_TS, torch.tensor(train_y_TS), K=K)


[tensor([-0.0038], device='cuda:0'), tensor([-0.1604,  0.2709, -0.0027, -0.0029, -0.0020, -0.0029, -0.0029, -0.0053,
        -0.0044, -0.0058, -0.0044, -0.0033, -0.0071, -0.0093, -0.0083, -0.0094,
        -0.0107], device='cuda:0'), tensor([-4.6577e-05,  1.3291e-04,  2.6433e-04,  ..., -7.7931e-05,
        -1.3400e-03, -6.7159e-04], device='cuda:0'), tensor([ 0.0006, -0.0005, -0.0010,  ..., -0.0007,  0.0057,  0.0031],
       device='cuda:0'), tensor([-0.4880], device='cuda:0'), tensor([-3.3250], device='cuda:0'), tensor([1.9112], device='cuda:0'), tensor([0.3002], device='cuda:0')]
det bpr: 0.18366919457912445
Pert bpr: 0.19162414968013763
nll: 3.6473419666290283
Loss: 4.455718040466309
EPOCH: 6
train_y_TS size: torch.Size([14, 1620])
ratio_rating_TS size: torch.Size([14, 1620])
[tensor([-0.0036], device='cuda:0'), tensor([-0.1536,  0.2594, -0.0026, -0.0028, -0.0019, -0.0028, -0.0028, -0.0051,
        -0.0043, -0.0057, -0.0043, -0.0032, -0.0070, -0.0091, -0.0082, -0.0092,
        -0.010